In [ ]:
!pip install google-generativeai python-dotenv omegaconf pytorch_lightning xformers diffusers==0.12.1 transformers==4.19.2 ftfy accelerate

In [ ]:
!pip install "jax[cuda12_pip]==0.4.23" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [26]:
from google.colab import output
from google.colab import userdata

from http.server import BaseHTTPRequestHandler, HTTPServer
import urllib.parse

# https://zenn.dev/peishim/articles/2e2e8408888f59
import os
from dotenv import load_dotenv
import google.generativeai as genai
import numpy as np
# https://gammasoft.jp/blog/stable-diffusion-with-diffusers-library/
import torch
from diffusers import StableDiffusionPipeline

In [25]:
PORT = 8088
PATH = ''

# from server.py

def gemini_setting():
    # .envファイルの読み込み
    load_dotenv()

    # API-KEYの設定

    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)

    model = genai.GenerativeModel("gemini-pro")
    # チャット履歴を初期化
    chat = model.start_chat(history=[])

    return chat

def conversation(chat, user):
    for i,hobby in enumerate(user):
        user_input = "{}人目の趣味は{}".format(i,hobby)
        response = chat.send_message(user_input)

    user_input2 = "彼らの趣味の共通点を英単語のみで言って"
    response2 = chat.send_message(user_input2)

    response2 = response2.text
    response2 = response2.split()
    #print(response2)

    user_input3 = "これらから単語を一つ選んで、選んだ単語だけ言って。ただし「*」,「Common」,「interests」,「hobbies」は選ばないで。"
    response3 = chat.send_message(user_input3)

    return response3.text

def ai_img(prompt):
    model_id = "CompVis/stable-diffusion-v1-4"
    device = "cuda"

    # パイプラインの作成
    pipe = StableDiffusionPipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16)
    pipe = pipe.to(device)

    # パイプラインの実行
    generator = torch.Generator(device).manual_seed(42) # seedを前回と同じ42にする
    with torch.autocast("cuda"):
        image = pipe(prompt, guidance_scale=7.5, generator=generator).images[0]

    # 生成した画像の保存
    image.save("ai_img.png")


class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):
    def do_GET(self):
      if self.path.startswith("/ai_img.png"):
        self.send_response(200)
        self.send_header('Content-type', 'image/png')
        self.end_headers()
        with open("./ai_img.png" ,"rb") as img:
          self.wfile.write(img.read())
      else:
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        with open("./submit.html", "rb") as file:
            self.wfile.write(file.read())

    def do_POST(self):
        # Content-Lengthヘッダーからデータサイズを取得
        content_length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(content_length)

        # 送信されたデータを解析
        parsed_data = urllib.parse.parse_qs(post_data.decode('utf-8'))
        ibody = parsed_data.get('ibody', [''])[0]

        user = []
        for i in ibody.splitlines():
            user.append(i)

        model = gemini_setting()
        keyword = conversation(model,user)

        # 解析結果を表示
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

        with open("./return.html","r") as f:
          response= (f.read())

        response+= keyword

        with open("./suff.html","r") as f:
          response+= (f.read())
        self.wfile.write(response.encode("utf-8"))

        # とりあえずタイムアウト回避に最後に生成
        ai_img(keyword)

# サーバー設定
server_address = ('', PORT)
httpd = HTTPServer(server_address, SimpleHTTPRequestHandler)

print(f"Server is ready! at")
output.serve_kernel_port_as_window(PORT, path=PATH)

httpd.serve_forever()

Server is ready! at :8088


127.0.0.1 - - [27/Oct/2024 05:27:27] "GET /?authuser=0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2024 05:27:28] "GET /favicon.ico?authuser=0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2024 05:28:43] "POST /?authuser=0 HTTP/1.1" 200 -


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

127.0.0.1 - - [27/Oct/2024 05:29:05] "GET /ai_img.png?authuser=0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2024 05:29:06] "GET /favicon.ico?authuser=0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2024 05:29:11] "GET /ai_img.png?1730006951085=&authuser=0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2024 05:29:16] "GET /ai_img.png?1730006956084=&authuser=0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2024 05:29:21] "GET /ai_img.png?1730006961084=&authuser=0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2024 05:29:26] "GET /ai_img.png?1730006966081=&authuser=0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2024 05:29:31] "GET /ai_img.png?1730006971083=&authuser=0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2024 05:29:36] "GET /ai_img.png?1730006976082=&authuser=0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2024 05:29:41] "GET /ai_img.png?1730006981081=&authuser=0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2024 05:29:46] "GET /ai_img.png?1730006986080=&authuser=0 HTTP/1.1" 200 -


KeyboardInterrupt: 